In [ ]:
# =============================
# Forward Selection RF Sweeps (Classification + Regression)
# =============================

# standard libs
import os, time, math, json, warnings
from urllib.parse import urlparse

# progress + display
from IPython.display import display

# data libs
import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt

# kaggle
import kagglehub

# sklearn
from sklearn.model_selection import (
    train_test_split, StratifiedKFold, KFold, StratifiedShuffleSplit, cross_val_score
)
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error
from sklearn.feature_selection import mutual_info_classif, f_regression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# clean prints
warnings.filterwarnings("ignore", category=UserWarning)
pd.options.display.float_format = "{:.6f}".format
np.set_printoptions(suppress=True)

# =============================
# Config
# =============================
RANDOM_STATE = 42
N_ROWS = 1_000_000  # cap when loading
DATASET_KEYS = ["steam", "olist", "sales"]  # sales == VG2019
CV_OUTER = 5       # for reported scores
CV_INNER = 3       # inside forward selection
K_VALUES = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
SAMPLE_SIZES = [10_000, 20_000, 30_000, 40_000, 50_000]
FORWARD_PREFILTER_TOP = 120  # shrink search pool before forward
EARLY_STOP_ROUNDS = 8        # stop if no gain for many steps
MIN_DELTA = 1e-6             # tiny improvement threshold

# =============================
# Small helpers
# =============================
def format_hms(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))

def list_csvs(folder_path):
    if folder_path is None or not os.path.exists(folder_path):
        return []
    return [f for f in os.listdir(folder_path) if f.lower().endswith(".csv")]

def try_read_csv(folder_path, file_name, **kwargs):
    full_path = os.path.join(folder_path, file_name) if folder_path is not None else None
    if full_path is not None and os.path.exists(full_path):
        return pd.read_csv(full_path, **kwargs)
    return None

def safe_kaggle_download(dataset_name):
    print(f"download: start {dataset_name}")
    t0 = time.perf_counter()
    try:
        path = kagglehub.dataset_download(dataset_name)
        print(f"download: ok -> {path} in {round(time.perf_counter() - t0, 2)} sec")
        return path
    except Exception as e:
        print(f"download: error -> {str(e)}")
        return None

def coerce_datetime_columns(df):
    if df is None:
        return None
    for col in df.columns:
        low = col.lower()
        if ("date" in low) or ("time" in low):
            try:
                df[col] = pd.to_datetime(df[col].astype("string"), errors="coerce")
            except Exception:
                pass
    return df

def optimize_dtypes(df, convert_categoricals=True, category_threshold=0.2):
    if df is None:
        return None
    start_mb = df.memory_usage(deep=True).sum() / (1024**2)
    for c in df.select_dtypes(include=[np.floating]).columns:
        df[c] = pd.to_numeric(df[c], downcast="float")
    for c in df.select_dtypes(include=[np.integer]).columns:
        df[c] = pd.to_numeric(df[c], downcast="integer")
    if convert_categoricals:
        obj_cols = [c for c in df.select_dtypes(include=["object"]).columns if ("date" not in c.lower() and "time" not in c.lower())]
        n_rows = max(len(df), 1)
        for c in obj_cols:
            try:
                if df[c].apply(lambda x: isinstance(x, (list, dict, set))).any():
                    continue
                uniq_ratio = df[c].nunique(dropna=False) / n_rows
                if uniq_ratio <= category_threshold:
                    df[c] = df[c].astype("category")
            except Exception:
                pass
    end_mb = df.memory_usage(deep=True).sum() / (1024**2)
    print(f"optimize_dtypes: {start_mb:.2f} MB -> {end_mb:.2f} MB")
    return df

# =============================
# Loaders (patched to avoid test_size=0)
# =============================
def load_steam_dataset(base_path, n_rows=1_000_000, seed=42):
    print("steam: start")
    if base_path is None:
        print("steam: skip (no path)")
        return None

    games = try_read_csv(base_path, "games.csv", low_memory=False)
    users = try_read_csv(base_path, "users.csv", low_memory=False)
    recs  = try_read_csv(base_path, "recommendations.csv", low_memory=False)

    meta = None
    meta_path = os.path.join(base_path, "games_metadata.json")
    if os.path.exists(meta_path):
        try:
            meta = pd.read_json(meta_path, lines=True)
        except Exception as e:
            print("steam: metadata read error:", str(e))

    print(f"steam: shapes games={None if games is None else games.shape}, users={None if users is None else users.shape}, recs={None if recs is None else recs.shape}, meta={None if meta is None else meta.shape}")

    if recs is None:
        print("steam: skip (recommendations.csv missing)")
        return None

    # patched: guard against test_size=0
    if "is_recommended" in recs.columns:
        total = len(recs)
        n_take = min(n_rows, total)
        if n_take >= total:
            recs = recs.copy()
        else:
            splitter = StratifiedShuffleSplit(n_splits=1, test_size=total - n_take, random_state=seed)
            keep_idx, _ = next(splitter.split(recs, recs["is_recommended"]))
            recs = recs.iloc[keep_idx].copy()
    else:
        recs = recs.sample(n=min(n_rows, len(recs)), random_state=seed).copy()

    if meta is not None and games is not None and "app_id" in meta.columns and "app_id" in games.columns:
        games = games.merge(meta, on="app_id", how="left")
    df = recs
    if games is not None and "app_id" in df.columns and "app_id" in games.columns:
        df = df.merge(games, on="app_id", how="left")
    if users is not None and "user_id" in df.columns and "user_id" in users.columns:
        df = df.merge(users, on="user_id", how="left")

    df = coerce_datetime_columns(df)
    print("steam: done shape", None if df is None else df.shape)
    return df

def load_olist_dataset(base_path, n_rows=1_000_000, seed=42):
    print("olist: start")
    if base_path is None:
        print("olist: skip (no path)")
        return None

    oc = try_read_csv(base_path, "olist_customers_dataset.csv")
    og = try_read_csv(base_path, "olist_geolocation_dataset.csv")
    oi = try_read_csv(base_path, "olist_order_items_dataset.csv")
    op = try_read_csv(base_path, "olist_order_payments_dataset.csv")
    orv = try_read_csv(base_path, "olist_order_reviews_dataset.csv")
    oo = try_read_csv(base_path, "olist_orders_dataset.csv")
    opr = try_read_csv(base_path, "olist_products_dataset.csv")
    osell = try_read_csv(base_path, "olist_sellers_dataset.csv")
    octr = try_read_csv(base_path, "product_category_name_translation.csv")

    print(
        "olist: shapes "
        f"customers={None if oc is None else oc.shape}, "
        f"geolocation={None if og is None else og.shape}, "
        f"items={None if oi is None else oi.shape}, "
        f"payments={None if op is None else op.shape}, "
        f"reviews={None if orv is None else orv.shape}, "
        f"orders={None if oo is None else oo.shape}, "
        f"products={None if opr is None else opr.shape}, "
        f"sellers={None if osell is None else osell.shape}, "
        f"cat_trans={None if octr is None else octr.shape}"
    )

    if not all(x is not None for x in [oo, oi, opr, osell, oc]):
        print("olist: skip (core tables missing)")
        return None

    orders_small = oo.sample(n=min(n_rows, len(oo)), random_state=seed)
    items_small = oi[oi["order_id"].isin(orders_small["order_id"])].copy()

    if octr is not None and "product_category_name" in opr.columns:
        products_en = opr.merge(octr, on="product_category_name", how="left")
    else:
        products_en = opr

    if orv is not None:
        pr = items_small[["order_id", "product_id"]].merge(orv[["order_id", "review_score"]], on="order_id", how="inner")
        pr = pr.drop_duplicates(["order_id", "product_id"])
        product_stats = pr.groupby("product_id", as_index=False).agg(
            review_count_product=("review_score", "count"),
            review_score_mean_product=("review_score", "mean"),
        )
    else:
        product_stats = None

    items_ext = (
        items_small.merge(products_en, on="product_id", how="left")
        .merge(osell, on="seller_id", how="left", suffixes=("", "_seller"))
    )

    if og is not None:
        geo_zip = og.groupby("geolocation_zip_code_prefix", as_index=False).agg(
            geolocation_lat=("geolocation_lat", "mean"),
            geolocation_lng=("geolocation_lng", "mean"),
            geo_points=("geolocation_city", "count"),
        )
        customers_geo = oc.merge(
            geo_zip,
            left_on="customer_zip_code_prefix",
            right_on="geolocation_zip_code_prefix",
            how="left",
        ).drop(columns=["geolocation_zip_code_prefix"])
    else:
        customers_geo = oc

    if op is not None:
        payments_agg = op.groupby("order_id", as_index=False).agg(
            payment_value_total=("payment_value", "sum"),
            payment_installments_max=("payment_installments", "max"),
            payment_count=("payment_type", "count"),
        )
    else:
        payments_agg = None

    olist_full = (
        orders_small.merge(customers_geo, on="customer_id", how="left")
        .merge(items_ext, on="order_id", how="left")
    )
    if payments_agg is not None:
        olist_full = olist_full.merge(payments_agg, on="order_id", how="left")
    if product_stats is not None:
        olist_full = olist_full.merge(product_stats, on="product_id", how="left")

    olist_full = coerce_datetime_columns(olist_full)
    print("olist: shape after assemble", olist_full.shape)
    return olist_full

def load_vg2019_dataset(base_path, n_rows=1_000_000, seed=42):
    print("vg2019: start")
    if base_path is None:
        print("vg2019: skip (no path)")
        return None
    csvs = list_csvs(base_path)
    target_csv = "vgsales-12-4-2019.csv" if "vgsales-12-4-2019.csv" in csvs else (csvs[0] if csvs else None)
    if target_csv is None:
        print("vg2019: skip (no csv)")
        return None
    df = pd.read_csv(os.path.join(base_path, target_csv), low_memory=False)
    print("vg2019: loaded", target_csv, "shape", df.shape)

    # patched: guard against test_size=0
    total = len(df)
    n_take = min(n_rows, total)
    if "Genre" in df.columns:
        if n_take >= total:
            df = df.copy()
        else:
            splitter = StratifiedShuffleSplit(n_splits=1, test_size=total - n_take, random_state=seed)
            keep_idx, _ = next(splitter.split(df, df["Genre"]))
            df = df.iloc[keep_idx].copy()
    else:
        df = df.sample(n=n_take, random_state=seed).copy()

    print("vg2019: done shape", df.shape)
    return df

# =============================
# Download + load
# =============================
print("main: start downloads")
steam_path = safe_kaggle_download("antonkozyriev/game-recommendations-on-steam")
olist_path = safe_kaggle_download("olistbr/brazilian-ecommerce")
vg2019_path = safe_kaggle_download("ashaheedq/video-games-sales-2019")
print("main: downloads finished")

t0_load = time.perf_counter()
steam = load_steam_dataset(steam_path, n_rows=N_ROWS, seed=RANDOM_STATE)
olist = load_olist_dataset(olist_path, n_rows=N_ROWS, seed=RANDOM_STATE)
sales = load_vg2019_dataset(vg2019_path, n_rows=N_ROWS, seed=RANDOM_STATE)
print(f"main: load all done in {round(time.perf_counter() - t0_load, 2)} sec ({format_hms(time.perf_counter() - t0_load)})")

if steam is not None:
    print("optimize steam")
    steam = optimize_dtypes(steam, True, 0.2)
if olist is not None:
    print("optimize olist")
    olist = optimize_dtypes(olist, True, 0.2)
if sales is not None:
    print("optimize vg2019")
    sales = optimize_dtypes(sales, True, 0.2)

# =============================
# Feature prep for both tasks
# =============================
def prepare_all(steam, olist, sales):
    print("prepare: start")

    # ----- Steam -----
    def prepare_steam(df_in):
        df = df_in.copy()

        # strings
        for c in ["title", "description", "tags"]:
            if c in df.columns:
                df[c] = df[c].astype("string")

        # dates
        def sdt(s): return pd.to_datetime(s.astype("string"), errors="coerce")
        for c in ["date", "date_release"]:
            df[c] = sdt(df[c]) if c in df.columns else pd.NaT

        # time parts
        df["days_since_release"] = (df["date"] - df["date_release"]).dt.days
        df["days_since_release"] = df["days_since_release"].clip(lower=0).fillna(0).astype("int32")
        df["review_year"] = df["date"].dt.year.fillna(-1).astype("int16")
        df["review_month"] = df["date"].dt.month.fillna(-1).astype("int8")
        df["review_dow"] = df["date"].dt.dayofweek.fillna(-1).astype("int8")

        # text lengths
        df["title_len"] = df["title"].str.len().fillna(0).astype("int32")
        df["desc_len"]  = df["description"].str.len().fillna(0).astype("int32")

        # numeric log1p
        for c in ["hours", "products", "reviews", "price_final", "price_original"]:
            df[c] = pd.to_numeric(df.get(c), errors="coerce")
            df[c + "_log1p"] = np.log1p(df[c])

        # platforms
        for b in ["win", "mac", "linux", "steam_deck"]:
            df[b] = (df.get(b) == True).astype("int8") if b in df.columns else 0

        # price flags
        df["is_free"] = (df["price_final"] == 0).astype("int8")
        df["discount_ratio"] = np.where(
            pd.to_numeric(df["price_original"], errors="coerce") > 0,
            1.0 - (pd.to_numeric(df["price_final"], errors="coerce") / pd.to_numeric(df["price_original"], errors="coerce")),
            0.0
        )
        df["discount_ratio"] = pd.Series(df["discount_ratio"]).clip(0, 1).fillna(0.0)

        # base features
        keep_dense = [
            "win","mac","linux","steam_deck",
            "days_since_release","review_year","review_month","review_dow",
            "title_len","desc_len",
            "hours_log1p","products_log1p","reviews_log1p",
            "price_final_log1p","price_original_log1p",
            "discount_ratio","is_free"
        ]
        X = df[keep_dense].copy()

        # top-K tags (binary)
        if "tags" in df.columns:
            print("steam: build tag one-hot")
            tags_clean = (
                df["tags"].astype("string").fillna("").str.lower()
                  .str.replace(r"[\[\]\"]", "", regex=True)
                  .str.replace(";", ",").str.replace("/", ",")
            )
            from sklearn.feature_extraction.text import CountVectorizer
            vec = CountVectorizer(
                tokenizer=lambda s: [t.strip() for t in s.split(",") if t.strip()],
                lowercase=False, binary=True, max_features=max(0, 200 - X.shape[1])
            )
            mat = vec.fit_transform(tags_clean.values)
            tag_df = pd.DataFrame(mat.toarray(), columns=[f"tag_{t}" for t in vec.get_feature_names_out()], index=df.index).astype("uint8")
            X = pd.concat([X, tag_df], axis=1)

        # numeric fix
        for c in X.select_dtypes(include=[np.number]).columns:
            X[c] = pd.to_numeric(X[c], errors="coerce").fillna(pd.to_numeric(X[c], errors="coerce").median())

        # targets
        pos = pd.to_numeric(df.get("positive_ratio"), errors="coerce")
        y_cls = (pos >= 80).astype("Int64")
        y_reg = pos.round().astype("Int64")

        # split by app_id time (avoid leakage)
        raw = df[["date", "app_id"]].copy()
        raw = raw.loc[X.index]
        raw["date"] = pd.to_datetime(raw["date"].astype("string"), errors="coerce")
        cutoff = raw["date"].sort_values().quantile(0.8)
        first = raw.groupby("app_id")["date"].min()
        app_train = set(first[first <= cutoff].index.tolist())
        app_test  = set(first[first >  cutoff].index.tolist())
        is_train = raw["app_id"].isin(app_train)
        is_test  = raw["app_id"].isin(app_test) | (~(raw["app_id"].isin(app_train) | raw["app_id"].isin(app_test)))

        def time_split(y):
            X_tr = X[is_train].reset_index(drop=True)
            X_te = X[is_test].reset_index(drop=True)
            y_tr = y[is_train].reset_index(drop=True)
            y_te = y[is_test].reset_index(drop=True)
            return X_tr, X_te, y_tr, y_te

        steam_cls_split = time_split(y_cls.dropna().reindex(X.index))
        steam_reg_split = time_split(y_reg.dropna().reindex(X.index))
        return steam_cls_split, steam_reg_split

    # ----- Olist -----
    def prepare_olist(df_in):
        o = df_in.copy()

        # dates -> parts, then drop
        def sdt(s): return pd.to_datetime(s.astype("string"), errors="coerce")
        date_cols = [
            "order_purchase_timestamp","order_approved_at","order_delivered_carrier_date",
            "order_delivered_customer_date","order_estimated_delivery_date","shipping_limit_date",
        ]
        for c in date_cols:
            o[c] = sdt(o[c]) if c in o.columns else pd.NaT
        o["purchase_dayofweek"] = o["order_purchase_timestamp"].dt.dayofweek
        o["purchase_month"] = o["order_purchase_timestamp"].dt.month
        o["purchase_hour"] = o["order_purchase_timestamp"].dt.hour

        def to_h(td): return td.dt.total_seconds() / 3600.0
        for c in ["product_length_cm","product_width_cm","product_height_cm","product_weight_g",
                  "payment_installments_max","payment_value_total","payment_count","freight_value","order_item_id"]:
            if c in o.columns: o[c] = pd.to_numeric(o[c], errors="coerce")
        o["approval_delay_h"] = to_h(o["order_approved_at"] - o["order_purchase_timestamp"])
        o["to_carrier_h"] = to_h(o["order_delivered_carrier_date"] - o["order_purchase_timestamp"])
        o["to_customer_h"] = to_h(o["order_delivered_customer_date"] - o["order_purchase_timestamp"])
        o["est_delivery_h"] = to_h(o["order_estimated_delivery_date"] - o["order_purchase_timestamp"])
        o["limit_from_purchase_h"] = to_h(o["shipping_limit_date"] - o["order_purchase_timestamp"])

        if "freight_value" not in o.columns:
            o["freight_value"] = np.nan
        o["product_volume_cm3"] = o["product_length_cm"] * o["product_width_cm"] * o["product_height_cm"]
        o["density_g_per_cm3"] = np.where(
            (o["product_volume_cm3"] > 0) & o["product_weight_g"].notna(),
            o["product_weight_g"] / o["product_volume_cm3"], np.nan
        )

        for c in ["payment_installments_max","payment_value_total","payment_count"]:
            if c not in o.columns: o[c] = np.nan
        o["avg_installment_value"] = np.where(
            pd.to_numeric(o["payment_installments_max"], errors="coerce") > 0,
            o["payment_value_total"] / o["payment_installments_max"], np.nan
        )
        o["payment_value_per_payment"] = np.where(
            pd.to_numeric(o["payment_count"], errors="coerce") > 0,
            o["payment_value_total"] / o["payment_count"], np.nan
        )

        if "order_item_id" not in o.columns: o["order_item_id"] = 1
        o["is_multi_item_order"] = (pd.to_numeric(o["order_item_id"], errors="coerce") > 1).astype("Int64")

        # category and states
        if "product_category_name_english" in o.columns:
            o["product_category"] = o["product_category_name_english"].astype("string")
        elif "product_category_name" in o.columns:
            o["product_category"] = o["product_category_name"].astype("string")
        else:
            o["product_category"] = "Unknown"

        cat_cols = []
        for c in ["order_status","customer_state","seller_state","product_category"]:
            if c in o.columns:
                o[c] = o[c].astype("string").fillna("Unknown")
                cat_cols.append(c)
        if cat_cols:
            o = pd.get_dummies(o, columns=cat_cols, dtype=np.uint8)

        # targets
        y_base = pd.to_numeric(df_in.get("review_score_mean_product"), errors="coerce")
        if y_base.notna().any():
            y_cls = (y_base >= 4.0).astype("Int64")
            y_reg = y_base.astype("Float64")
        else:
            late = (o.get("order_delivered_customer_date") > o.get("order_estimated_delivery_date")).astype("Int64")
            y_cls = late
            y_reg = late.astype("Float64")

        # drop ids and raw dates
        drop_ids = ["order_id","customer_id","customer_unique_id","product_id","seller_id"]
        for c in drop_ids:
            if c in o.columns: o = o.drop(columns=[c])
        for c in date_cols:
            if c in o.columns: o = o.drop(columns=[c])

        # numeric clean
        for c in o.select_dtypes(include=[np.number]).columns:
            o[c] = pd.to_numeric(o[c], errors="coerce").fillna(pd.to_numeric(o[c], errors="coerce").median())

        # split
        def split(y, stratify=False):
            X = o.copy()
            if stratify:
                X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
            else:
                X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
            return X_tr.reset_index(drop=True), X_te.reset_index(drop=True), y_tr.reset_index(drop=True), y_te.reset_index(drop=True)

        olist_cls_split = split(y_cls.dropna(), stratify=True)
        olist_reg_split = split(y_reg.dropna(), stratify=False)
        return olist_cls_split, olist_reg_split

    # ----- VG2019 -----
    def prepare_sales(df_in):
        s = df_in.copy()

        # targets
        if "Critic_Score" in s.columns:
            cs = pd.to_numeric(s["Critic_Score"], errors="coerce")
            y_cls = (cs > 8.0).astype("Int64")
            y_reg = cs.astype("Float64")
        else:
            y_cls = pd.Series(np.nan, index=s.index, dtype="Int64")
            y_reg = pd.Series(np.nan, index=s.index, dtype="Float64")

        # remove leaks and junk
        leak = ["NA_Sales","PAL_Sales","JP_Sales","Other_Sales","Total_Shipped","Rank","Global_Sales","Critic_Score"]
        junk = ["VGChartz_Score","Vgchartzscore","url","img_url","status","Last_Update","basename","User_Score","Name"]
        s = s.drop(columns=[c for c in leak + junk if c in s.columns], errors="ignore")

        # platform family + handheld
        def platform_family(p):
            p = "" if pd.isna(p) else str(p).upper()
            if p.startswith("PS") or p in {"PSP","PSV"}: return "PlayStation"
            if p.startswith("X") or p in {"XB","XBLA"}: return "Xbox"
            if p in {"SWITCH","WII","WIIU","GC","N64","SNES","NES","DS","3DS","GB","GBC","GBA"}: return "Nintendo"
            if p == "PC": return "PC"
            if p in {"DC","DREAMCAST","SAT","GEN","MD","MEGADRIVE","GG"}: return "Sega"
            if "ATARI" in p: return "Atari"
            return "Other"
        if "Platform" in s.columns:
            s["Platform_Family"] = s["Platform"].apply(platform_family)
            handhelds = {"DS","3DS","GB","GBC","GBA","PSP","PSV"}
            s["is_portable"] = s["Platform"].astype("string").str.upper().isin(handhelds).astype("Int64")
            s = s.drop(columns=["Platform"])
        else:
            s["Platform_Family"] = "Other"
            s["is_portable"] = pd.Series(0, index=s.index, dtype="Int64")

        # year + decade
        if "Year" in s.columns:
            s["Year"] = pd.to_numeric(s["Year"], errors="coerce")
            s.loc[~s["Year"].between(1970, 2025, inclusive="both"), "Year"] = np.nan
            s["Decade"] = (s["Year"] // 10 * 10).astype("Int64").astype(str)
        else:
            s["Year"] = np.nan
            s["Decade"] = "<NA>"

        # frequency encodes
        for col in ["Publisher","Developer"]:
            if col in s.columns:
                ser = s[col].astype("string").fillna("Unknown")
                freq = ser.map(ser.value_counts(normalize=True))
                s[col + "_freq"] = freq.astype(float)
                s = s.drop(columns=[col])

        # one-hot
        for c in ["Genre","ESRB_Rating","Platform_Family","Decade"]:
            if c in s.columns:
                s[c] = s[c].astype("string").fillna("Unknown")
        s = pd.get_dummies(s, columns=[c for c in ["Genre","ESRB_Rating","Platform_Family","Decade"] if c in s.columns], dtype=np.uint8)

        # numeric clean
        for c in s.select_dtypes(include=[np.number]).columns:
            s[c] = pd.to_numeric(s[c], errors="coerce").fillna(pd.to_numeric(s[c], errors="coerce").median())

        # split
        def split(y, stratify=False):
            X = s.copy()
            if stratify:
                X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
            else:
                X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
            return X_tr.reset_index(drop=True), X_te.reset_index(drop=True), y_tr.reset_index(drop=True), y_te.reset_index(drop=True)

        sales_cls_split = split(y_cls.dropna(), stratify=True)
        sales_reg_split = split(y_reg.dropna(), stratify=False)
        return sales_cls_split, sales_reg_split

    steam_cls, steam_reg = prepare_steam(steam)
    olist_cls, olist_reg = prepare_olist(olist)
    sales_cls, sales_reg = prepare_sales(sales)

    # simple scaling (no polys here to keep it simple)
    def scale_split(split):
        X_tr, X_te, y_tr, y_te = split
        num_cols = X_tr.select_dtypes(include=[np.number]).columns.tolist()
        if len(num_cols) == 0:
            return split
        scaler = StandardScaler()
        tr_scaled = scaler.fit_transform(X_tr[num_cols].astype("float32"))
        te_scaled = scaler.transform(X_te[num_cols].astype("float32"))
        X_tr2 = X_tr.copy(); X_te2 = X_te.copy()
        X_tr2[num_cols] = tr_scaled
        X_te2[num_cols] = te_scaled
        return (X_tr2, X_te2, y_tr, y_te)

    return {
        "classification": {
            "steam": scale_split(steam_cls),
            "olist": scale_split(olist_cls),
            "sales": scale_split(sales_cls),
        },
        "regression": {
            "steam": scale_split(steam_reg),
            "olist": scale_split(olist_reg),
            "sales": scale_split(sales_reg),
        }
    }

print("prepare: building splits...")
PREP = prepare_all(steam, olist, sales)
print("prepare: done")

# =============================
# Build full X,y for each task
# =============================
def get_full_xy(task_type):
    data = PREP[task_type]
    out = {}
    for key in DATASET_KEYS:
        X_tr, X_te, y_tr, y_te = data[key]
        X_all = pd.concat([X_tr, X_te], axis=0, ignore_index=True)
        y_all = pd.concat([pd.Series(y_tr), pd.Series(y_te)], axis=0, ignore_index=True)
        out[key] = (X_all, y_all)
    return out

def stratified_sample_xy(X, y, n, seed):
    if n >= len(X):
        return X.copy(), y.copy()
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=len(X)-n, random_state=seed)
    keep_idx, _ = next(splitter.split(X, y))
    return X.iloc[keep_idx].reset_index(drop=True), y.iloc[keep_idx].reset_index(drop=True)

def simple_sample_xy(X, y, n, seed):
    if n >= len(X):
        return X.copy(), y.copy()
    rs = np.random.RandomState(seed)
    keep_idx = rs.choice(len(X), size=n, replace=False)
    return X.iloc[keep_idx].reset_index(drop=True), y.iloc[keep_idx].reset_index(drop=True)

# =============================
# True forward selection
# =============================
def prefilter_features(X, y, top_p, task_type):
    p = min(int(top_p), X.shape[1])
    if p <= 0:
        return X.columns[:0]
    if task_type == "classification":
        scores = mutual_info_classif(X, y, random_state=RANDOM_STATE)
        order = np.argsort(scores)[::-1][:p]
    else:
        sc, _ = f_regression(X, y)
        sc = np.nan_to_num(sc, nan=0.0, neginf=0.0, posinf=np.nanmax(sc) if np.isfinite(np.nanmax(sc)) else 0.0)
        order = np.argsort(sc)[::-1][:p]
    return X.columns[order]

def forward_select_order(X, y, max_k, task_type, inner_folds=3, prefilter_top=120, early_stop_rounds=8, min_delta=1e-6):
    t0 = time.perf_counter()
    pool_cols = prefilter_features(X, y, prefilter_top, task_type)
    Xp = X[pool_cols].reset_index(drop=True)

    selected = []
    remain = list(Xp.columns)
    no_improve = 0

    if task_type == "classification":
        base_est = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1)
        scoring = "f1_macro"
        last_best = -1e9
    else:
        base_est = RandomForestRegressor(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1)
        scoring = "neg_root_mean_squared_error"
        last_best = 1e9  # store RMSE (lower is better)

    while remain and len(selected) < int(max_k):
        best_feat = None
        best_score = None
        for feat in remain:
            cols_now = selected + [feat]
            if task_type == "classification":
                cv = StratifiedKFold(n_splits=inner_folds, shuffle=True, random_state=RANDOM_STATE)
            else:
                cv = KFold(n_splits=inner_folds, shuffle=True, random_state=RANDOM_STATE)
            scores = cross_val_score(base_est, Xp[cols_now], y, cv=cv, scoring=scoring, n_jobs=-1)
            mean_score = float(scores.mean())
            if task_type == "regression":
                # convert to positive RMSE
                mean_score = -mean_score
                if (best_score is None) or (mean_score < best_score):
                    best_score = mean_score
                    best_feat = feat
            else:
                if (best_score is None) or (mean_score > best_score):
                    best_score = mean_score
                    best_feat = feat

        selected.append(best_feat)
        remain.remove(best_feat)

        if task_type == "classification":
            gain = best_score - (last_best if last_best != -1e9 else 0.0)
            last_best = best_score
            if gain <= min_delta:
                no_improve += 1
            else:
                no_improve = 0
        else:
            gain = (last_best - best_score) if last_best != 1e9 else 1e9
            last_best = best_score
            if gain <= min_delta:
                no_improve += 1
            else:
                no_improve = 0

        if no_improve >= int(early_stop_rounds):
            print(f"forward: early stop at k={len(selected)} (no gain for {early_stop_rounds} steps)")
            break

    t_sel = time.perf_counter() - t0
    return selected, t_sel, list(pool_cols)

FORWARD_CACHE = {"classification": {}, "regression": {}}

def get_forward_order_cached(task_type, dataset_key, Xn, yn, max_k):
    key = (dataset_key, len(Xn))
    if key in FORWARD_CACHE[task_type]:
        return FORWARD_CACHE[task_type][key]
    order, t_sel, pool = forward_select_order(
        Xn, yn, max_k=max_k, task_type=task_type,
        inner_folds=CV_INNER, prefilter_top=FORWARD_PREFILTER_TOP,
        early_stop_rounds=EARLY_STOP_ROUNDS, min_delta=MIN_DELTA
    )
    FORWARD_CACHE[task_type][key] = (order, t_sel, pool)
    return FORWARD_CACHE[task_type][key]

# =============================
# Run sweeps
# =============================
def run_sweep(task_type, sample_sizes, k_values, seed):
    xy = get_full_xy(task_type)
    rows = []
    for dataset_key in DATASET_KEYS:
        X_all, y_all = xy[dataset_key]
        for n in sample_sizes:
            if task_type == "classification":
                Xn, yn = stratified_sample_xy(X_all, y_all, n, seed)
            else:
                Xn, yn = simple_sample_xy(X_all, y_all, n, seed)

            print(f"[{task_type}] forward order • {dataset_key} • n={len(Xn)} • prefilter={FORWARD_PREFILTER_TOP} • inner_cv={CV_INNER}")
            order, time_select, pre_cols = get_forward_order_cached(task_type, dataset_key, Xn, yn, max(K_VALUES))

            for k in k_values:
                k = int(min(k, len(order)))
                if k == 0:
                    continue
                cols_use = order[:k]
                t0 = time.perf_counter()

                if task_type == "classification":
                    cv = StratifiedKFold(n_splits=CV_OUTER, shuffle=True, random_state=seed)
                    model = RandomForestClassifier(n_estimators=300, random_state=seed, n_jobs=-1)
                    f1_list = []
                    for tr_idx, te_idx in cv.split(Xn[cols_use], yn):
                        Xtr, Xte = Xn[cols_use].iloc[tr_idx], Xn[cols_use].iloc[te_idx]
                        ytr, yte = yn.iloc[tr_idx], yn.iloc[te_idx]
                        model.fit(Xtr, ytr)
                        y_pred = model.predict(Xte)
                        f1_list.append(f1_score(yte, y_pred, average="macro"))
                    time_cv = time.perf_counter() - t0
                    row = {
                        "dataset": dataset_key,
                        "sample_size": int(len(Xn)),
                        "k_features": k,
                        "f1_macro": float(np.mean(f1_list)),
                        "time_select_sec_total": float(time_select),
                        "time_cv_sec": float(time_cv),
                        "time_total_sec": float(time_select + time_cv)
                    }
                else:
                    cv = KFold(n_splits=CV_OUTER, shuffle=True, random_state=seed)
                    model = RandomForestRegressor(n_estimators=300, random_state=seed, n_jobs=-1)
                    mae_list, rmse_list = [], []
                    for tr_idx, te_idx in cv.split(Xn[cols_use], yn):
                        Xtr, Xte = Xn[cols_use].iloc[tr_idx], Xn[cols_use].iloc[te_idx]
                        ytr, yte = yn.iloc[tr_idx], yn.iloc[te_idx]
                        model.fit(Xtr, ytr)
                        y_pred = model.predict(Xte)
                        mae_list.append(mean_absolute_error(yte, y_pred))
                        rmse_list.append(math.sqrt(mean_squared_error(yte, y_pred)))
                    time_cv = time.perf_counter() - t0
                    row = {
                        "dataset": dataset_key,
                        "sample_size": int(len(Xn)),
                        "k_features": k,
                        "mae": float(np.mean(mae_list)),
                        "rmse": float(np.mean(rmse_list)),
                        "time_select_sec_total": float(time_select),
                        "time_cv_sec": float(time_cv),
                        "time_total_sec": float(time_select + time_cv)
                    }

                rows.append(row)
                nice = {k: v for k, v in row.items() if k not in ["dataset","sample_size","k_features"]}
                print(f"[{task_type}] {dataset_key} | n={row['sample_size']} | k={row['k_features']} -> {nice}")
    return pd.DataFrame(rows)

# =============================
# Plot helpers
# =============================
def plot_metric_vs_k(df, task_type):
    metric = "f1_macro" if task_type == "classification" else "rmse"
    name = "F1 Macro" if task_type == "classification" else "RMSE"
    for dataset_key in DATASET_KEYS:
        sub = df[df["dataset"] == dataset_key].copy()
        if sub.empty:
            continue
        plt.figure(figsize=(7, 5))
        for n in sorted(sub["sample_size"].unique()):
            part = sub[sub["sample_size"] == n]
            means = part.groupby("k_features")[metric].mean().reset_index()
            plt.plot(means["k_features"], means[metric], marker="o", label=f"n={n}")
        plt.title(f"{dataset_key.upper()} • {name} vs. #Features (forward)")
        plt.xlabel("#Features (k)")
        plt.ylabel(name)
        plt.legend()
        plt.grid(True)
        plt.show()

def plot_time_vs_samples(df, task_type):
    for dataset_key in DATASET_KEYS:
        sub = df[df["dataset"] == dataset_key].copy()
        if sub.empty:
            continue
        plt.figure(figsize=(7, 5))
        for k in sorted(sub["k_features"].unique()):
            part = sub[sub["k_features"] == k]
            means = part.groupby("sample_size")["time_total_sec"].mean().reset_index()
            plt.plot(means["sample_size"], means["time_total_sec"], marker="o", label=f"k={k}")
        plt.title(f"{dataset_key.upper()} • Total Time vs. Sample Size")
        plt.xlabel("Sample size")
        plt.ylabel("Time (sec)")
        plt.legend()
        plt.grid(True)
        plt.show()

def pick_best_rows(df, task_type):
    best_rows = []
    for dataset_key in DATASET_KEYS:
        sub = df[df["dataset"] == dataset_key].copy()
        if sub.empty:
            continue
        idx = sub["f1_macro"].idxmax() if task_type == "classification" else sub["rmse"].idxmin()
        best_rows.append(sub.loc[idx])
    return pd.DataFrame(best_rows).reset_index(drop=True)

def refit_and_plot_top_features(task_type, dataset_key, sample_size, k):
    xy = get_full_xy(task_type)
    X_all, y_all = xy[dataset_key]
    if task_type == "classification":
        Xn, yn = stratified_sample_xy(X_all, y_all, sample_size, RANDOM_STATE)
        order, _, _ = get_forward_order_cached("classification", dataset_key, Xn, yn, max(K_VALUES))
        model = RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE, n_jobs=-1)
    else:
        Xn, yn = simple_sample_xy(X_all, y_all, sample_size, RANDOM_STATE)
        order, _, _ = get_forward_order_cached("regression", dataset_key, Xn, yn, max(K_VALUES))
        model = RandomForestRegressor(n_estimators=300, random_state=RANDOM_STATE, n_jobs=-1)

    k = int(min(k, len(order)))
    cols = order[:k]
    model.fit(Xn[cols], yn)
    fi = getattr(model, "feature_importances_", None)
    if fi is None:
        return None
    order_idx = np.argsort(fi)[::-1]
    top = min(20, len(order_idx))
    top_idx = order_idx[:top]
    top_names = [cols[i] for i in top_idx]
    top_vals = fi[top_idx]

    plt.figure(figsize=(8, 5))
    plt.barh(range(top), top_vals[::-1])
    plt.yticks(range(top), [top_names[i] for i in range(top - 1, -1, -1)])
    plt.title(f"{dataset_key.upper()} • Top Features ({task_type}, k={k}, n={sample_size})")
    plt.xlabel("Importance")
    plt.ylabel("Feature")
    plt.tight_layout()
    plt.show()

    return pd.DataFrame({"feature": top_names, "importance": top_vals})

# =============================
# Run experiments
# =============================
print("starting classification sweep (forward)...")
cls_df = run_sweep("classification", SAMPLE_SIZES, K_VALUES, RANDOM_STATE)
print("classification sweep: done.\n")

print("starting regression sweep (forward)...")
reg_df = run_sweep("regression", SAMPLE_SIZES, K_VALUES, RANDOM_STATE)
print("regression sweep: done.\n")

# =============================
# Tables
# =============================
print("\n=== Classification (RandomForest + true forward selection) ===")
display(cls_df.sort_values(["dataset","sample_size","k_features"]).reset_index(drop=True))

print("\n=== Regression (RandomForest + true forward selection) ===")
display(reg_df.sort_values(["dataset","sample_size","k_features"]).reset_index(drop=True))

# =============================
# Graphs
# =============================
plot_metric_vs_k(cls_df, "classification")
plot_metric_vs_k(reg_df, "regression")

plot_time_vs_samples(cls_df, "classification")
plot_time_vs_samples(reg_df, "regression")

# =============================
# Best combos + feature bars
# =============================
best_cls = pick_best_rows(cls_df, "classification")
best_reg = pick_best_rows(reg_df, "regression")

print("\n=== Best settings (Classification) ===")
display(best_cls)

print("\n=== Best settings (Regression) ===")
display(best_reg)

cls_importance_tables = {}
reg_importance_tables = {}

for _, r in best_cls.iterrows():
    ds, n, k = str(r["dataset"]), int(r["sample_size"]), int(r["k_features"])
    tab = refit_and_plot_top_features("classification", ds, n, k)
    cls_importance_tables[ds] = tab

for _, r in best_reg.iterrows():
    ds, n, k = str(r["dataset"]), int(r["sample_size"]), int(r["k_features"])
    tab = refit_and_plot_top_features("regression", ds, n, k)
    reg_importance_tables[ds] = tab

# =============================
# Summary
# =============================
def summarize(df, task_type):
    lines = []
    for _, r in df.iterrows():
        ds = str(r["dataset"]).upper()
        n = int(r["sample_size"])
        k = int(r["k_features"])
        tsel = float(r["time_select_sec_total"])
        tcv = float(r["time_cv_sec"])
        if task_type == "classification":
            f1m = float(r["f1_macro"])
            lines.append(f"{ds}: best k={k}, n={n}, F1={f1m:.4f}, select={tsel:.1f}s, cv={tcv:.1f}s")
        else:
            mae = float(r["mae"])
            rmse = float(r["rmse"])
            lines.append(f"{ds}: best k={k}, n={n}, RMSE={rmse:.4f}, MAE={mae:.4f}, select={tsel:.1f}s, cv={tcv:.1f}s")
    return "\n".join(lines)

print("\n================ SUMMARY (true forward selection) ================\n")
print("Classification (higher F1 is better):")
print(summarize(best_cls, "classification"))
print("\nRegression (lower RMSE and MAE are better):")
print(summarize(best_reg, "regression"))
print("\nImportance tables in:")
print(" - cls_importance_tables['steam'|'olist'|'sales']")
print(" - reg_importance_tables['steam'|'olist'|'sales']")
